# Writing the json files 

In [1]:
import sys
import os
sys.path.append('/dls/science/groups/e02/Mohsen/code/Git_Repos/Merlin-Medipix/')
import epsic_tools.api as epsic

In [2]:
sim_matrix_path = '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_matrix_ptyREX_amC_16June2020_512pixArray/'

In [3]:
dirs_to_prep = epsic.sim_utils.get_raw_dir_list(sim_matrix_path, get_all = True)

In [4]:
iteration_number = 2000
for path in dirs_to_prep:
    it = os.scandir(path)
    for entry in it:
        if entry.is_file():
            if entry.name.endswith('txt'):
                entry_path = '/'+os.path.join(*entry.path.split('/')[:-1])
                for item in os.listdir(entry_path):
                    if item.endswith('h5'):
                        params_file = os.path.join(entry_path, entry)
                        h5_file = os.path.join(entry_path, item)
                        exp_dict = epsic.sim_utils.parse_params_file(params_file, h5_file)
                        epsic.sim_utils.write_ptyrex_json(exp_dict, iteration_number)
#                         print(os.path.join(entry_path, item))

In [18]:
# # To delete json files
# json_files = []
# for path in dirs_to_prep:
#     it = os.scandir(path)
#     for entry in it:
#         if entry.is_file():
#             if entry.name.endswith('json'):
#                 json_files.append(entry.path)
# for json_file in json_files:
#     os.remove(json_file)

# Running ptyREX

In [5]:
dirs_to_run_ptyREX = epsic.sim_utils.get_ptyREX_ready(sim_matrix_path) 

In [6]:
script_path = '/dls/science/groups/e02/Mohsen/code/Git_Repos/Staff-notebooks/ptyREX_sim_matrix'

In [7]:
len(dirs_to_run_ptyREX)

64

In [8]:
for path in dirs_to_run_ptyREX:
    it = os.scandir(path)
    for entry in it:
        if entry.is_file():
            if entry.name.startswith('ptyREX'):
                output_folder = os.path.dirname(entry.path)
                json_file = os.path.splitext(entry.name)[0]
                os.system('\n cd '+ output_folder + '\n module load global/cluster \n qsub '+ script_path + '/ptyREX_batch_submit.sh '+ output_folder + ' ' + json_file + ' 2000iter')